#### 데이터 로드

In [3]:
import pandas as pd


In [4]:
data = pd.read_csv('UniversalBank.csv')

print(type(data))
print(data.shape)
print(data.info())
print(data)

data.head()



<class 'pandas.core.frame.DataFrame'>
(5000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 5000 non-null   int64  
 1   Age                5000 non-null   int64  
 2   Experience         5000 non-null   int64  
 3   Income             5000 non-null   int64  
 4   ZIPCode            5000 non-null   int64  
 5   Family             5000 non-null   int64  
 6   CCAvg              5000 non-null   float64
 7   Education          5000 non-null   int64  
 8   Mortgage           5000 non-null   int64  
 9   PersonalLoan       5000 non-null   int64  
 10  SecuritiesAccount  5000 non-null   int64  
 11  CDAccount          5000 non-null   int64  
 12  Online             5000 non-null   int64  
 13  CreditCard         5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB
None
        ID  Age  E

,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,PersonalLoan,SecuritiesAccount,CDAccount,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


- PersonalLoan 이 대출 가능여부 클래스
- ID, ZIPCode 는 필요 없어보이므로 드롭

#### x,y 데이터 나누기

In [5]:
x_data = data.drop(['ID','ZIPCode','PersonalLoan'],axis=1)
y_data = data['PersonalLoan']

print(type(x_data))
print(x_data.shape)
print(x_data)

print(type(y_data))
print(y_data.shape)
print(y_data)

<class 'pandas.core.frame.DataFrame'>
(5000, 11)
      Age  Experience  Income  ...  CDAccount  Online  CreditCard
0      25           1      49  ...          0       0           0
1      45          19      34  ...          0       0           0
2      39          15      11  ...          0       0           0
3      35           9     100  ...          0       0           0
4      35           8      45  ...          0       0           1
...   ...         ...     ...  ...        ...     ...         ...
4995   29           3      40  ...          0       1           0
4996   30           4      15  ...          0       1           0
4997   63          39      24  ...          0       0           0
4998   65          40      49  ...          0       1           0
4999   28           4      83  ...          0       1           1

[5000 rows x 11 columns]
<class 'pandas.core.series.Series'>
(5000,)
0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0

#### train, test 셋 분할

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.4, random_state=1)

print(type(x_train))
print(x_train.shape)
print(type(x_test))
print(x_test.shape)
print(type(y_train))
print(y_train.shape)
print(type(y_test))
print(y_test.shape)

<class 'pandas.core.frame.DataFrame'>
(3000, 11)
<class 'pandas.core.frame.DataFrame'>
(2000, 11)
<class 'pandas.core.series.Series'>
(3000,)
<class 'pandas.core.series.Series'>
(2000,)


#### 모델 앙상블

- voting 방식

In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3,p=2)

from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression(random_state=1, C=1.0, solver='lbfgs')

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=1,criterion='entropy', max_depth=5)

In [11]:
from sklearn.ensemble import VotingClassifier

voting = VotingClassifier(estimators=[
        ('knn', knn),  ('tree', tree)], voting='soft') #  voting='soft' 는 단순 다수결이 아닌 확률기반 voting='hard'는 단순 다수결
# eclf1 = eclf1.fit(x_train, y_train)


#### 교차검증

In [12]:
from sklearn.model_selection import cross_val_score

clf_label =['knn','tree','voting'] # 분류기 라벨
clf_model =[knn,tree,voting] # 분류기 모델

# 반복문으로 각 분류기 교차 검증
# voting 앙상블만 사용하려면 반복문 불필요
for clf,label in zip(clf_model,clf_label):
  scores = cross_val_score(estimator = clf,X=x_train,y=y_train, cv=10) # 모델,데이터,cv는 crossvalidation으로 k겹 수 
  print(scores.mean(),scores.std(),label)



0.9063333333333332 0.01797838208021574 knn
0.9876666666666665 0.006674994798166919 tree
0.9743333333333334 0.005174724898753328 voting


- 로지스틱회귀가 오류나서 일단 제외